# Ildebrando Magnani

## Econ - Problem Set 4 

### Exercise 2:

In [1]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as lin
import scipy.integrate as intgr
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from math import gamma
from scipy.special import beta

In [2]:
macro_data = np.loadtxt('MacroSeries.txt', delimiter = ',')
C_t = macro_data[:,0]
K_t = macro_data[:,1]
W_t = macro_data[:,2]
R_t = macro_data[:,3]

In [3]:
beta = 0.99

### a):

In [4]:
def Z_series(K_series, W_series, alpha):
    
    Z_series = np.log(W_series) - alpha * np.log(K_series) - np.log(1-alpha)
    
    return Z_series


def log_lik(K_series, W_series, params):
    
    alpha, rho, mu, sigma = params
    log_pdf_list = []
    
    Z = Z_series(K_series, W_series, alpha)
    Z = np.insert(Z, 0, mu)
    
    for t, z in  enumerate(Z):
        
        if t > 0:
            
            mean = rho * Z[t-1] + (1-rho) * mu
            pdf_t = sts.norm.pdf(Z[t], loc=(mean), scale=(sigma**2))
             
            if pdf_t == 0:
                
                pdf_t = 1e-10
            
            log_pdf_t = np.log(pdf_t)
            log_pdf_list.append(log_pdf_t)
            
    log_pdf_array = np.asarray(log_pdf_list)
    log_lik = np.sum(log_pdf_array)
    
    return log_lik   


def crit(params, K_series, W_series):
    
    LL = log_lik(K_series, W_series, params)
    
    return -LL

In [5]:
alpha_0 = 0.35
rho_0   = 0.5
mu_0    = 6
sigma_0 = 0.4

params_init = (alpha_0, rho_0, mu_0, sigma_0)
MLE_args = (K_t, W_t)

results = opt.minimize(crit, params_init, args=(MLE_args), method='L-BFGS-B', 
                            bounds=((1e-10, 0.9999999), (-0.9999999, 0.9999999), (1e-10, None), (1e-10, None)))

In [6]:
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results.x
Hess_inv = results.hess_inv
MLE_params = (alpha_MLE, rho_MLE, mu_MLE, sigma_MLE)
print('alpha_MLE =',alpha_MLE, ', rho_MLE =', rho_MLE, ', mu_MLE =', mu_MLE, ', sigma_MLE =', sigma_MLE)
print('Log-Likelihood function evaluated at MLE parameters:', log_lik(K_t, W_t, MLE_params))
print('Hessian Inverse:')
print(Hess_inv.todense())

alpha_MLE = 0.999594175287 , rho_MLE = 0.24993562826 , mu_MLE = 8.15022859748 , sigma_MLE = 0.311385006955
Log-Likelihood function evaluated at MLE parameters: 91.444770178
Hessian Inverse:
[[  3.10710394e+00  -1.19190499e+03   7.64832960e+03  -8.20311948e+01]
 [ -1.19190499e+03   4.57222872e+05  -2.93394826e+06   3.14677203e+04]
 [  7.64832960e+03  -2.93394826e+06   1.88268390e+07  -2.01924888e+05]
 [ -8.20311948e+01   3.14677203e+04  -2.01924888e+05   2.16572172e+03]]


### b):

In [7]:
def Z_series1(R_series, K_series, alpha):
    
    Z_series = np.log(R_series) - (alpha-1) * np.log(K_series) - np.log(alpha)
    
    return Z_series


def log_lik1(R_series, K_series, params):
    
    alpha, rho, mu, sigma = params
    log_pdf_list = []
    
    Z = Z_series1(R_series, K_series, alpha)
    Z = np.insert(Z, 0, mu)
    
    for t, z in  enumerate(Z):
        
        if t > 0:
            
            mean = rho * Z[t-1] + (1-rho) * mu
            pdf_t = sts.norm.pdf(Z[t], loc=(mean), scale=(sigma**2))
             
            if pdf_t == 0:
                
                pdf_t = 1e-10
            
            log_pdf_t = np.log(pdf_t)
            log_pdf_list.append(log_pdf_t)
            
    log_pdf_array = np.asarray(log_pdf_list)
    log_lik = np.sum(log_pdf_array)
    
    return log_lik   


def crit1(params, R_series, K_series):
    
    LL = log_lik1(R_series, K_series, params)
    neg_LL = -LL
    
    return neg_LL

In [20]:
params_init1 = (0.5, 0.5, 1, 10)
MLE_args1 = (R_t, K_t)

results1 = opt.minimize(crit1, params_init1, args=(MLE_args1), method='L-BFGS-B', 
                            bounds=((1e-10, 0.9999999), (-0.9999999, 0.9999999), (1e-10, None), (1e-10, None)))

In [24]:
alpha_MLE1, rho_MLE1, mu_MLE1, sigma_MLE1 = results1.x
Hess_inv1 = results1.hess_inv
MLE_params1 = (alpha_MLE1, rho_MLE1, mu_MLE1, sigma_MLE1)
print('alpha_MLE =',alpha_MLE1, ', rho_MLE =', rho_MLE1, ', mu_MLE =', mu_MLE1, ', sigma_MLE =', sigma_MLE1)
print('Log-Likelihood function evaluated at MLE parameters:', log_lik1(R_t, K_t, MLE_params1))
print('Hessian Inverse:')
print(Hess_inv1.todense())

alpha_MLE = 0.701976707439 , rho_MLE = 0.479970670942 , mu_MLE = 5.07600524872 , sigma_MLE = 0.303386860758
Log-Likelihood function evaluated at MLE parameters: 96.6537371947
Hessian Inverse:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


### c):

In [34]:
k_t = 7500000
z_t_minus = 10
z_star = (alpha_MLE-1) * np.log(k_t) - np.log(alpha_MLE)

zt_cdf = lambda z: sts.norm.cdf(z, loc=(rho_MLE * z_t_minus + (1-rho_MLE) * mu_MLE), scale=(sigma_MLE**2))

Prob = 1 - zt_cdf(z_star)
print('Probability interest rate > 1 = ', Prob)

Probability interest rate > 1 =  1.0
